<a href="https://colab.research.google.com/github/jseeun00/ESAA_2021-2/blob/main/13%EC%A3%BC%EC%B0%A8_%ED%95%84%EC%82%AC1_7_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CH7. 앙상블 학습과 랜덤 포레스트**

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

##**7.1 투표 기반 분류기**

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [3]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.904


##**7.2 배깅과 페이스팅**

In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [5]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500, bootstrap=True, n_jobs=-1, oob_score = True)

In [6]:
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.8986666666666666

In [7]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.912

In [8]:
bag_clf.oob_decision_function_

array([[0.44508671, 0.55491329],
       [0.37288136, 0.62711864],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.1299435 , 0.8700565 ],
       [0.37128713, 0.62871287],
       [0.01142857, 0.98857143],
       [0.98816568, 0.01183432],
       [0.97191011, 0.02808989],
       [0.71584699, 0.28415301],
       [0.00531915, 0.99468085],
       [0.75384615, 0.24615385],
       [0.81065089, 0.18934911],
       [0.94086022, 0.05913978],
       [0.06214689, 0.93785311],
       [0.        , 1.        ],
       [0.98876404, 0.01123596],
       [0.95336788, 0.04663212],
       [1.        , 0.        ],
       [0.015     , 0.985     ],
       [0.3045977 , 0.6954023 ],
       [0.92178771, 0.07821229],
       [1.        , 0.        ],
       [0.97814208, 0.02185792],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.63333333, 0.36666667],
       [0.

##**7.4. 랜덤 포레스트**

In [9]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [10]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(max_features="auto",max_leaf_nodes=16),
                           n_estimators = 500, max_samples=1.0, bootstrap=True, n_jobs=-1)

In [11]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators = 500, n_jobs = -1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name,score)

sepal length (cm) 0.09706288184492083
sepal width (cm) 0.023409093045636348
petal length (cm) 0.45522977297992856
petal width (cm) 0.4242982521295144


##**7.5 부스팅**

In [12]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

In [13]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [14]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

In [15]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

In [17]:
#y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [18]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120,)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)

gbrt_best = GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=107)

In [20]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break  # 조기 종료

In [21]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train,y_train)
y_pred = xgb_reg.predict(X_val)

[16:00:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [22]:
xgb_reg.fit(X_train, y_train, eval_set = [(X_val,y_val)],early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

[16:00:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.467425
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.439855
[2]	validation_0-rmse:0.415817
[3]	validation_0-rmse:0.395314
[4]	validation_0-rmse:0.377963
[5]	validation_0-rmse:0.361861
[6]	validation_0-rmse:0.34866
[7]	validation_0-rmse:0.337979
[8]	validation_0-rmse:0.32774
[9]	validation_0-rmse:0.31916
[10]	validation_0-rmse:0.312899
[11]	validation_0-rmse:0.306829
[12]	validation_0-rmse:0.301491
[13]	validation_0-rmse:0.297583
[14]	validation_0-rmse:0.293627
[15]	validation_0-rmse:0.290212
[16]	validation_0-rmse:0.287413
[17]	validation_0-rmse:0.285059
[18]	validation_0-rmse:0.282925
[19]	validation_0-rmse:0.278406
[20]	validation_0-rmse:0.277246
[21]	validation_0-rmse:0.272575
[22]	validation_0-rmse:0.270854
[23]	validation_0-rmse:0.267572
[24]	validation_0-rmse:0.266329
[25]	validation_0-rmse